In [1]:
!pip install -qU \
  langchain-core  \
  langchain[google-genai]  \
  langchain-community  \
  langsmith  \
  google-search-results 

## Tools

In [2]:
from langchain_core.tools import tool

@tool
def add(x: float, y:float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def subtract(x: float, y:float) -> float:
    """Subtract 'x' and 'y'."""
    return x - y

@tool
def multiply(x: float, y:float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def divide(x: float, y:float) -> float:
    """Divide 'x' and 'y'."""
    return x / y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y



In [3]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7f6528305990>)

In [4]:
print(f"{add.name}\n{add.description}\n{add.args_schema.model_json_schema()}")

add
Add 'x' and 'y'.
{'description': "Add 'x' and 'y'.", 'properties': {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}, 'required': ['x', 'y'], 'title': 'add', 'type': 'object'}


In [5]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [6]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

{'x': 5, 'y': 2}

In [7]:
exponentiate.func(**llm_output_dict)

25

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [9]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ········


In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)

/tmp/ipykernel_38038/2322671580.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, divide, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [12]:
agent.invoke({
    "input": "What is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps":[]
})


[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"x": 10.7, "y": 7.68}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ebe0f63c-e81b-4a02-a505-cb9bf7c82faf-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'd5a0e06e-f56b-4504-8dac-7f8a51a5e99a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 268, 'output_tokens': 109, 'total_tokens': 377, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 85}})], tool_call_id='d5a0e06e-f56b-4504-8dac-7f8a51a5e99a')]

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

In [14]:
agent_executor.invoke({
    "input": "What is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.7 multiplied by 7.68 is 82.176.

> Finished chain.


{'input': 'What is 10.7 multiplied by 7.68',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})],
 'output': '10.7 multiplied by 7.68 is 82.176.'}

In [15]:
memory.chat_memory.messages

[HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
 AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})]

In [16]:
agent_executor.invoke({
    "input": "My name is Muhammad",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...
Hello Muhammad! It's nice to meet you.

> Finished chain.


{'input': 'My name is Muhammad',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Muhammad', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Muhammad! It's nice to meet you.", additional_kwargs={}, response_metadata={})],
 'output': "Hello Muhammad! It's nice to meet you."}

In [17]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2 to the power of 3",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 2.0, 'y': 3.0}`


8.0
Invoking: `multiply` with `{'x': 4.0, 'y': 8.0}`


32.0Nine plus 10 is 19. 4 multiplied by 2 to the power of 3 is 32. So, 19 minus 32 is -13.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2 to the power of 3',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Muhammad', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Muhammad! It's nice to meet you.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2 to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nine plus 10 is 19. 4 multiplied by 2 to the power of 3 is 32. So, 19 minus 32 is -13.', additional_kwargs={}, response_metadata={})],
 'output': 'Nine plus 10 is 19. 4 multiplied by 2 to the power of 3 is 32. So, 19 minus 32 is -13.'}

In [18]:
agent_executor.invoke({
    "input": "9+10-4*2**3",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 2.0, 'y': 3.0}`


8.0
Invoking: `multiply` with `{'x': 4.0, 'y': 8.0}`


32.09 + 10 - 4 * 2 to the power of 3 equals -13.

> Finished chain.


{'input': '9+10-4*2**3',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Muhammad', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Muhammad! It's nice to meet you.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2 to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nine plus 10 is 19. 4 multiplied by 2 to the power of 3 is 32. So, 19 minus 32 is -13.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='9+10-4*2**3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='9 + 10 - 4 * 2 to the power of 3 equals -13.', additional_kwargs={}, response_metadata={})],
 'output': '9 + 10 - 4 * 2 to the power of 3 equals -13.'}

In [19]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...
Your name is Muhammad.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Muhammad', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Muhammad! It's nice to meet you.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2 to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nine plus 10 is 19. 4 multiplied by 2 to the power of 3 is 32. So, 19 minus 32 is -13.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='9+10-4*2**3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='9 + 10 - 4 * 2 to the power of 3 equals -13.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_meta

### API Calling

In [24]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get Geographical Location from IP"""
    
    try:
        response = requests.get("https://ipinfo.io/json")
        
        data = response.json()
        if "loc" in data:
            latitude, longitude = data["loc"].split(",")
        
        data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
        )
        return data
    except Exception as e:
        return f"Error Occured {e}"

@tool
def get_current_datetime():
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [25]:
print(f"{get_location_from_ip.name}\n{get_location_from_ip.description}\n{get_location_from_ip.args_schema.model_json_schema()}")
print(f"{get_current_datetime.name}\n{get_current_datetime.description}\n{get_current_datetime.args_schema.model_json_schema()}")
 

get_location_from_ip
Get Geographical Location from IP
{'description': 'Get Geographical Location from IP', 'properties': {}, 'title': 'get_location_from_ip', 'type': 'object'}
get_current_datetime
Return the current date and time.
{'description': 'Return the current date and time.', 'properties': {}, 'title': 'get_current_datetime', 'type': 'object'}


In [28]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistance"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [31]:
tools = [get_location_from_ip, get_current_datetime]

agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools, 
    verbose = True
)



In [34]:
agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "Do you know where I am"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2025-07-29 16:50:00
Invoking: `get_location_from_ip` with `{}`


Latitude: 42.3314,
Longitude: -83.0457,
City: Detroit,
Country: USThe current date and time is 2025-07-29 16:50:00. You are in Detroit, US.

> Finished chain.


{'input': 'I have a few questions, what is the date and time right now? Do you know where I am',
 'output': 'The current date and time is 2025-07-29 16:50:00. You are in Detroit, US.'}